Codes to transfer the muon reco and trigger efficiency scale factors from json format to root format. It would create the root file with the same name in the same folder as the json file.

In [1]:
import json
import numpy as np
import ROOT as rt

Welcome to JupyROOT 6.14/09


# Reco efficiency

In [11]:
class RecoSF:
    def __init__(self,name):
        self.file_name=name
        self.hist_name=""
        self.data=None
        self.__ReadJSON()
        
    def __ReadJSON(self):
        f=open(self.file_name)
        a=json.load(f)
        self.hist_name=a.keys()[0].encode('utf-8')
        a=a[a.keys()[0]]
        a=a[a.keys()[0]]
        self.data=a
        
    def __toBin(self,key):
        s=key.encode('utf-8')
        left=float(s.split("[")[-1].split(",")[0])
        right=float(s.split("]")[0].split(",")[-1])
        return left,right
    
    def __toBins(self,var):
        binDict=self.data[self.data.keys()[0]]
        for each in binDict:
            if each['variable'].encode('utf-8')==var:
                return np.array(each['binning'])
        print("Variable {0} not found!".format(var))
        
    def JSONtoROOT(self):
        pt_bins=self.__toBins('pt')
        eta_bins=self.__toBins('abseta')
        hist=rt.TH2F(self.hist_name+"_abseta_pt",";|#eta|;p_{T} (GeV)",len(eta_bins)-1,eta_bins.astype(float),len(pt_bins)-1,pt_bins.astype(float))
        hist_stat=rt.TH2F(self.hist_name+"_abseta_pt_stat",";|#eta|;p_{T} (GeV)",len(eta_bins)-1,eta_bins.astype(float),len(pt_bins)-1,pt_bins.astype(float))
        hist_syst=rt.TH2F(self.hist_name+"_abseta_pt_syst",";|#eta|;p_{T} (GeV)",len(eta_bins)-1,eta_bins.astype(float),len(pt_bins)-1,pt_bins.astype(float))

        for ipt in range(len(pt_bins)-1):
            for ieta in range(len(eta_bins)-1):
                pt_key=u"pt:[{0},{1}]".format(pt_bins[ipt],pt_bins[ipt+1])
                eta_key=u"abseta:[{0},{1}]".format(eta_bins[ieta],eta_bins[ieta+1])
                nom=self.data[eta_key][pt_key][u'value']
                stat=self.data[eta_key][pt_key][u'stat']
                syst=self.data[eta_key][pt_key][u'syst']
                hist.SetBinContent(ieta+1,ipt+1,nom)
                hist_stat.SetBinContent(ieta+1,ipt+1,nom)
                hist_stat.SetBinError(ieta+1,ipt+1,stat)
                hist_syst.SetBinContent(ieta+1,ipt+1,nom)
                hist_syst.SetBinError(ieta+1,ipt+1,syst)

        f_out=rt.TFile(self.file_name.replace("json","root"),"recreate")
        f_out.cd()
        hist.Write()
        hist_stat.Write()
        hist_syst.Write()
        f_out.Write()
        f_out.Close()

In [12]:
recoSF=RecoSF("NUM_TrackerMuons_DEN_genTracks_Z_abseta_pt.json")
recoSF.JSONtoROOT()

# Trigger efficiency

In [38]:
class TrgSF:
    def __init__(self,name):
        self.file_name=name
        self.hist_name={}
        self.data={}
        self.nInputs={}
        self.nScheme=0
        self.__ReadJSON()
        
    def __ReadJSON(self):
        f=open(self.file_name)
        a=json.load(f)['corrections']
        self.nScheme=len(a)
        for iS in range(self.nScheme):
            self.hist_name[iS]=a[iS]['name'].encode('utf-8')
            self.nInputs[iS]=len(a[iS]['inputs'])-1
            self.data[iS]=a[iS]
            
    def __GetHist2D(self,i):
        data=self.data[i]
        var1=data['inputs'][0]['name'].encode('utf-8')
        var2=data['inputs'][1]['name'].encode('utf-8')
        data=data['data']
        bins1=np.array(data['edges'])
        bins2=np.array(data['content'][0]['edges'])
        nbin1=len(bins1)-1
        nbin2=len(bins2)-1
        hist_name=self.hist_name[i]+"_"+var1+"_"+var2
        hist=rt.TH2F(hist_name,";|#eta|;p_{T} (GeV)",nbin1,bins1,nbin2,bins2)
        hist_stat=rt.TH2F(hist_name+"_stat",";|#eta|;p_{T} (GeV)",nbin1,bins1,nbin2,bins2)
        hist_syst=rt.TH2F(hist_name+"_syst",";|#eta|;p_{T} (GeV)",nbin1,bins1,nbin2,bins2)
        for i1 in range(nbin1):
            for i2 in range(nbin2):
                nom=data['content'][i1]['content'][i2]['content'][5]['value']
                stat=data['content'][i1]['content'][i2]['content'][3]['value']
                syst=data['content'][i1]['content'][i2]['content'][4]['value']
                hist.SetBinContent(i1+1,i2+1,nom)
                hist_stat.SetBinContent(i1+1,i2+1,nom)
                hist_stat.SetBinError(i1+1,i2+1,stat)
                hist_syst.SetBinContent(i1+1,i2+1,nom)
                hist_syst.SetBinError(i1+1,i2+1,syst)
        return hist,hist_stat,hist_syst
        
    def __GetHist3D(self,i):
        data=self.data[i]
        var1=data['inputs'][0]['name'].encode('utf-8')
        var2=data['inputs'][1]['name'].encode('utf-8')
        var3=data['inputs'][2]['name'].encode('utf-8')
        data=data['data']
        bins1=np.array(data['edges'])
        bins2=np.array(data['content'][0]['edges'])
        bins3=np.array(data['content'][0]['content'][0]['edges'])
        nbin1=len(bins1)-1
        nbin2=len(bins2)-1
        nbin3=len(bins3)-1
        hist_name=self.hist_name[i]+"_"+var1+"_"+var2+"_"+var3
        hist=rt.TH3F(hist_name,"charge;|#eta|;p_{T} (GeV)",nbin1,bins1,nbin2,bins2,nbin3,bins3)
        hist_stat=rt.TH3F(hist_name+"_stat",";charge;|#eta|;p_{T} (GeV)",nbin1,bins1,nbin2,bins2,nbin3,bins3)
        hist_syst=rt.TH3F(hist_name+"_syst",";charge;|#eta|;p_{T} (GeV)",nbin1,bins1,nbin2,bins2,nbin3,bins3)
        for i1 in range(nbin1):
            for i2 in range(nbin2):
                for i3 in range(nbin3):
                    nom=data['content'][i1]['content'][i2]['content'][i3]['content'][5]['value']
                    stat=data['content'][i1]['content'][i2]['content'][i3]['content'][3]['value']
                    syst=data['content'][i1]['content'][i2]['content'][i3]['content'][4]['value']
                    hist.SetBinContent(i1+1,i2+1,i3+1,nom)
                    hist_stat.SetBinContent(i1+1,i2+1,i3+1,nom)
                    hist_stat.SetBinError(i1+1,i2+1,i3+1,stat)
                    hist_syst.SetBinContent(i1+1,i2+1,i3+1,nom)
                    hist_syst.SetBinError(i1+1,i2+1,i3+1,syst)
        return hist,hist_stat,hist_syst
        
    def JSONtoROOT(self):
        hist={}
        hist_stat={}
        hist_syst={}
        for iS in range(self.nScheme):
            if self.nInputs[iS]==2:
                hist[iS],hist_stat[iS],hist_syst[iS]=self.__GetHist2D(iS)
            elif self.nInputs[iS]==3:
                hist[iS],hist_stat[iS],hist_syst[iS]=self.__GetHist3D(iS)
            else:
                print("Number of inputs {0} not correct!!!".format(self.nInputs[iS]))
                return
        
        f_out=rt.TFile(self.file_name.replace("json","root"),"recreate")
        for iS in range(self.nScheme):
            hist[iS].Write()
            hist_stat[iS].Write()
            hist_syst[iS].Write()
        f_out.Write()
        f_out.Close()

In [39]:
trgSF=TrgSF("Efficiencies_muon_generalTracks_Z_Run2016_UL_HIPM_SingleMuonTriggers_schemaV2.json")
trgSF.JSONtoROOT()